In [4]:
from pyomo.environ import *


In [5]:
# Crear el modelo
model = ConcreteModel()

In [6]:

# Parámetros constantes
model.T = RangeSet(1, 24)  # Horizonte de 24 horas
model.ICC = Param(initialize=50)  # Costo marginal
model.Heat_Price = Param(initialize=38.13)  # Precio anual fijo de calor (ejemplo: 2024)
model.Bmax = Param(initialize=1500)  # Capacidad máxima del buffer
model.CP = Param(initialize=1000)  # Costo de partida (ejemplo)
model.aFRR_up = Param(initialize=10)  # Reservas aFRR positivas
model.aFRR_down = Param(initialize=10)  # Reservas aFRR negativas
model.Pmin = Param(initialize=100)  # Potencia mínima
model.Pmax = Param(initialize=300)  # Potencia máxima
model.Mercado_spot = Param(model.T, initialize=lambda model, t: 100 + t)  # Ejemplo de precios spot horarios
model.Demanda_q = Param(model.T, initialize=lambda model, t: 500)  # Demanda térmica fija (puede variar)



In [ ]:
# Variables
model.P = Var(model.T, domain=NonNegativeReals, bounds=(0, model.Pmax))  # Potencia generada
model.Q = Var(model.T, domain=NonNegativeReals)  # Calor generado
model.B = Var(model.T, domain=NonNegativeReals, bounds=(0, model.Bmax))  # Buffer de calor
model.Costo_partida = Var(model.T, domain=NonNegativeReals)  # Costos de partida
model.ENS = Var(model.T, domain=NonNegativeReals)  # Energía no servida
model.is_starting = Var(model.T, domain=Binary)  # Variable binaria para encendido

# Función objetivo
def objective_rule(model):
    return sum(
        model.P[t] * (model.Mercado_spot[t] - model.ICC) +  # Ingreso por venta de electricidad
        model.Heat_Price * model.Demanda_q[t] -  # Ingreso por venta de calor
        model.Costo_partida[t] -  # Costo de partida
        model.ENS[t]  # Energía no servida
        for t in model.T
    )
model.obj = Objective(rule=objective_rule, sense=maximize)




In [12]:
# Restricciones

# Restricción de potencia generada con reservas aFRR
def power_limits_rule(model, t):
    return inequality(
        model.Pmin + model.aFRR_up,
        model.P[t],
        model.Pmax - model.aFRR_down
    )
model.power_limits = Constraint(model.T, rule=power_limits_rule)

# Restricción de buffer (calor almacenado)
def buffer_balance_rule(model, t):
    if t == 1:
        return model.B[t] == model.Q[t] - model.Demanda_q[t]
    return model.B[t] == model.B[t - 1] + model.Q[t] - model.Demanda_q[t]
model.buffer_balance = Constraint(model.T, rule=buffer_balance_rule)

# Restricción para energía no servida (ENS)
def energy_not_served_rule(model, t):
    return model.ENS[t] >= 4 * model.ICC * (model.Demanda_q[t] - model.Q[t] - model.B[t - 1])
model.energy_not_served = Constraint(model.T, rule=energy_not_served_rule)

# Restricción de costos de partida
def startup_cost_rule(model, t):
    return model.Costo_partida[t] == model.is_starting[t] * model.CP
model.startup_cost = Constraint(model.T, rule=startup_cost_rule)

# Restricciones de tiempo mínimo de encendido y apagado
def min_off_time_rule(model, t):
    if t >= 4:
        return sum(model.is_starting[t - k] for k in range(4)) <= 1
    return Constraint.Skip
model.min_off_time = Constraint(model.T, rule=min_off_time_rule)

def min_on_time_rule(model, t):
    if t >= 4:
        return sum(model.is_starting[t - k] for k in range(4)) >= 1
    return Constraint.Skip
model.min_on_time = Constraint(model.T, rule=min_on_time_rule)

# Resolver el problema
solver = SolverFactory('glpk')  # Puedes usar 'cbc' o 'glpk'
result = solver.solve(model, tee=True)

# Mostrar resultados
print("Resultado:", result)
for t in model.T:
    print(f"Hora {t}: P = {model.P[t].value}, Q = {model.Q[t].value}, B = {model.B[t].value}, ENS = {model.ENS[t].value}")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
